In [12]:
import networkx as nx
import osmnx as ox
import folium

### Importing Required Libraries
This code imports three essential libraries:
1. **NetworkX (`nx`)**: For working with graphs and networks.
2. **OSMnx (`ox`)**: For downloading and analyzing street networks.
3. **Folium**: For creating interactive maps and visualizations.

In [13]:
# Download the street network of Kathmandu
city_graph = ox.graph.graph_from_place("Kathmandu, Nepal", network_type="drive")
city_graph = ox.routing.add_edge_speeds(city_graph)
city_graph = ox.routing.add_edge_travel_times(city_graph)

### Downloading and Preparing the Street Network
This code downloads the street network of **Kathmandu, Nepal** using OSMnx. It then adds:
1. **Edge Speeds**: To include vehicle speeds on the roads.
2. **Edge Travel Times**: To estimate how long it takes to travel along each road segment.

These steps prepare the network for route analysis and further exploration.

In [23]:
# Define start, end, and optional waypoint coordinates
start_coords = (27.69970737488004, 85.35676239625207)  # Tribhuvan International Airport
end_coords = (27.733461006817382, 85.30841816824604)      # Gongabu Bus Park
waypoint_coords = (27.706081413150063, 85.3076778310561)       # Basantapur Durbar Square

### Defining Start, End, and Waypoint Coordinates
This code sets the coordinates for three key locations in Kathmandu:
1. **Start Point**: Tribhuvan International Airport.
2. **End Point**: Gongabu Bus Park.
3. **Waypoint**: Basantapur Durbar Square (optional intermediate stop).

These coordinates will be used to calculate and visualize the routes.

In [24]:
#Find the nearest nodes
start_node = ox.distance.nearest_nodes(city_graph, X=start_coords[1], Y=start_coords[0])
end_node = ox.distance.nearest_nodes(city_graph, X=end_coords[1], Y=end_coords[0])
waypoint_node = ox.distance.nearest_nodes(city_graph, X=waypoint_coords[1], Y=waypoint_coords[0])

### Finding the Nearest Nodes on the Street Network
This code finds the nearest nodes (intersections) on the street network for the following locations:
1. **Start Point**: The nearest node to Tribhuvan International Airport.
2. **End Point**: The nearest node to Gongabu Bus Park.
3. **Waypoint**: The nearest node to Basantapur Durbar Square.

These nodes represent the closest points on the road network to the given coordinates.

In [25]:
# Compute routes
# Shortest travel time route
shortest_route_time = ox.routing.shortest_path(city_graph, start_node, end_node, weight="travel_time")
# Shortest distance route
shortest_route_distance = ox.routing.shortest_path(city_graph, start_node, end_node, weight="length")
# Route via waypoint
route_with_waypoint = (
    ox.routing.shortest_path(city_graph, start_node, waypoint_node, weight="travel_time") +
    ox.routing.shortest_path(city_graph, waypoint_node, end_node, weight="travel_time")[1:]
)   

### Computing Routes
This code computes three types of routes between the start and end points:
1. **Shortest Travel Time Route**: The route that minimizes travel time, using "travel_time" as the weight.
2. **Shortest Distance Route**: The route that minimizes the total distance, using "length" as the weight.
3. **Route via Waypoint**: A route that passes through an intermediate waypoint (Basantapur Durbar Square), combining two shortest paths: from start to waypoint and from waypoint to end.


In [26]:
# Create an interactive map
route_map = folium.Map(location=[(start_coords[0] + end_coords[0]) / 2, (start_coords[1] + end_coords[1]) / 2],
                       zoom_start=14, tiles="cartodbpositron")

### Creating an Interactive Map
This code creates an interactive map using **Folium**:
- The map is centered at the midpoint between the start and end coordinates.
- The **zoom level** is set to 14 for better street-level visibility.
- The **CartoDB positron** style is used for a clean, minimal map design.

This map will be used to visualize the routes on the map.

In [27]:
# Add routes to the map
folium.PolyLine([(city_graph.nodes[node]["y"], city_graph.nodes[node]["x"]) for node in shortest_route_time],
                color="blue", weight=5, opacity=0.7, tooltip="Shortest Time Route").add_to(route_map)

folium.PolyLine([(city_graph.nodes[node]["y"], city_graph.nodes[node]["x"]) for node in shortest_route_distance],
                color="green", weight=5, opacity=0.7, tooltip="Shortest Distance Route").add_to(route_map)

folium.PolyLine([(city_graph.nodes[node]["y"], city_graph.nodes[node]["x"]) for node in route_with_waypoint],
                color="orange", weight=5, opacity=0.7, tooltip="Route via Waypoint").add_to(route_map)


### Adding Routes to the Map
This code adds the computed routes to the interactive map:
1. **Shortest Time Route**: Displayed in blue with a tooltip showing "Shortest Time Route".
2. **Shortest Distance Route**: Displayed in green with a tooltip showing "Shortest Distance Route".
3. **Route via Waypoint**: Displayed in orange with a tooltip showing "Route via Waypoint".

Each route is represented as a **polyline** connecting the nodes of the respective path on the street network.


In [28]:
# Add start, end, and waypoint markers
folium.Marker(location=start_coords, tooltip="Start", icon=folium.Icon(color="green")).add_to(route_map)
folium.Marker(location=end_coords, tooltip="End", icon=folium.Icon(color="red")).add_to(route_map)
folium.Marker(location=waypoint_coords, tooltip="Waypoint", icon=folium.Icon(color="purple")).add_to(route_map)

In [29]:
# Display travel metrics
# Shortest route length (time)
route_length_time = sum(ox.routing.route_to_gdf(city_graph, shortest_route_time)["length"])
route_time = sum(ox.routing.route_to_gdf(city_graph, shortest_route_time)["travel_time"]) / 60  # in minutes
folium.Marker(location=[start_coords[0], start_coords[1] + 0.002], 
              popup=f"Shortest Time Route: {int(route_length_time)} m, {route_time:.1f} min").add_to(route_map)

### Displaying Travel Metrics for the Shortest Time Route
This code calculates and displays the travel metrics for the **Shortest Time Route**:
- **Route Length**: The total distance in meters.
- **Travel Time**: The total time in minutes, converted from seconds.

The metrics are displayed on the map as a **popup** on the start marker, showing both the route length and travel time.

In [30]:
# Shortest route length (distance)
route_length_distance = sum(ox.routing.route_to_gdf(city_graph, shortest_route_distance)["length"])
folium.Marker(location=[start_coords[0] + 0.002, start_coords[1]],
              popup=f"Shortest Distance Route: {int(route_length_distance)} m").add_to(route_map)

### Displaying Travel Metrics for the Shortest Distance Route
This code calculates and displays the travel metrics for the **Shortest Distance Route**:
- **Route Length**: The total distance in meters.

The metric is displayed on the map as a **popup** on the start marker, showing the route length.


In [ ]:
# Add legend HTML
legend_html = """
    <div style="position: fixed; 
                top: 10px; bottom: 10px; right: 10px; width: 180px; height: 160px; 
                background-color: white; z-index: 9999; font-size: 12px; 
                border:2px solid grey; border-radius: 5px; padding: 10px;">
        <b>Route Legend</b><br><br>
        <i style="background:blue; padding:5px;"></i> Shortest Time Route<br>
        <i style="background:green; padding:5px;"></i> Shortest Distance Route<br>
        <i style="background:orange; padding:5px;"></i> Route via Waypoint<br>
        <i style="background:green; padding:5px;"></i> Start<br>
        <i style="background:red; padding:5px;"></i> End<br>
        <i style="background:purple; padding:5px;"></i> Waypoint<br>
    </div>
"""
route_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map
route_map
